# Web Scraping (Static Website)

- **Static websites** are fixed and display the *same content for every user*, usually written exclusively in HTML and CSS. They don't provide any specialized content revision depending on user preference. 


- Internet communication involves a **server** and a **web browser**. To establish a connection between the two, a set of rules called **Hypertext Transfer Protocol (HTTP)** is used. Simply put, 
    1. The *web browser transmits an HTTP request to the server*
    
    2. The *server replies with an HTTP response along with the requested webpage in HTML*.


- Static websites usually come with a *fixed number of pages that have a specific layout*. When the page runs on a browser, **the content is literally static and doesn’t change in response to user actions**. A static website is usually created with HTML and CSS in simple text editors like Notepad.


- Even though the website will display the same thing with no intricate navigation details, static websites don’t need to feature just plain text. An HTML website can look beautiful, but the page’s **source code won’t change, no matter what actions a user takes on it.**


- The following code examples are modification on top of [Web Scraping with Python - Beautiful Soup Crash Course](https://www.youtube.com/watch?v=XVv6mJpFOb0)

## Basic Functionality
- The following code requests a **STATIC WEBSITE** that has different job postings that have the *company name*, *published date*, *skills needed*, and a reference to where to get *more information*.


- The basic functionality of this code is to traverse over job postings and extract the necessary information. We're mostly interested in getting the latest job postings, so we filter the ones that have the `a few days ago` for the published date.

- Important implementation points are

    1. `html_text = requests.get('https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=Python&txtLocation=').text`
       - The `.text` tag is very important. If we don't specifically ask for the text, meaning that we ommit it, we'll get the **status code** of the request.
       
    2. `published_date = job.find('span', class_="sim-posted").span.text.strip()`
        - The `.span` is used here because the *published_date* is wrapped in two `<span></span>` tags. By just typing `.span`, we can shortcut the process and directly extract the text.

    3. ` more_info = job.header.h2.a['href']`
        - Similar to the previous implementation, here we do `.header` to get the header of the job container, then `.h2` to get into the `<h2></h2>` tag, and finally put `a['href']` to specifically get the **link** associated with the more information.
        

In [ ]:
from bs4 import BeautifulSoup
import requests

html_text = requests.get('https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=Python&txtLocation=').text

soup = BeautifulSoup(html_text, 'lxml')

jobs = soup.find_all('li', class_="clearfix job-bx wht-shd-bx")
for job in jobs:
    published_date = job.find('span', class_="sim-posted").span.text.strip()
    if 'few' in published_date:
        company_name = job.find('h3', class_="joblist-comp-name").text.replace(' ', '')
        skills_needed = job.find('span', class_="srp-skills").text.replace(' ', '')
        # the link to the full-description of the job is under header-->h2-->a tag
        # with the link being under the "href"
        more_info = job.header.h2.a['href']

        print(f"Company Name: {company_name.strip()}")
        print(f"Required Skills: {skills_needed.strip()}")
        print(f"More Information: {more_info}")

        print('')

## Filtering Data
- **Filtering Modification with Unfamiliar Skills**


- The user is prompted to input the `unfamiliar_skills` that he has. These `unfamiliar_skills` are then scanned in each job description, only *jobs that don't have any of these `unfamiliar_skills`* are returned.


- **Key Point:** The *set* implementation in Python is useful here. Specifically the following line provides us with this functionality
    - `if(len(set(skills) & set(unfamiliar_skills)) == 0):`

In [ ]:
from bs4 import BeautifulSoup
import requests

print('What are the skills you\'re not familiar with?')
print('When you\'re done, press ENTER:')
unfamiliar_skills = []
skill = input('>>')
while(skill != ''):
    unfamiliar_skills.append(skill)
    skill = input('>>')

print(f'Filtering out {unfamiliar_skills} from job search')


html_text = requests.get('https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=Python&txtLocation=').text

soup = BeautifulSoup(html_text, 'lxml')

jobs = soup.find_all('li', class_="clearfix job-bx wht-shd-bx")
for job in jobs:
    published_date = job.find('span', class_="sim-posted").span.text.strip()
    if 'few' in published_date:
        company_name = job.find('h3', class_="joblist-comp-name").text.replace(' ', '').strip()
        skills_needed = job.find('span', class_="srp-skills").text.replace(' ', '').strip()

        skills = skills_needed.split(',')
        if(len(set(skills) & set(unfamiliar_skills)) == 0):

        # the link to the full-description of the job is under header-->h2-->a tag
        # with the link being under the "href"
            more_info = job.header.h2.a['href']

            print(f"Company Name: {company_name}")
            print(f"Required Skills: {skills_needed}")
            print(f"More Information: {more_info}")

            print('')

## Automating 
- **Automation of Job Scraping**

- After the user is prompted for filtering out `unfamiliar_skills`, the whole scraping project is automated to run with a **specified time intervals**. Here, this time is *30 minutes* and is done with a call to `time.sleep(sleep_time * 60)`. 
    1. After each scraping process, the scraped jobs are written to a file with the format `jobs_version[version]` inside the directory `jobs_pool`.
```python
with open(f'jobs_pool/jobs_version{version}.txt', 'w') as f:
    find_jobs(f, version)
# run every 30 minutes
sleep_time = 30
print(f"Waiting for {sleep_time} minutes ...")
time.sleep(sleep_time * 60)
```
    2. The jobs are written into this `.txt` file with the format
```python
f.write(f"Company Name: {company_name} \n")
f.write(f"Required Skills: {skills_needed} \n")
f.write(f"More Information: {more_info} \n")
f.write('\n')
```

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

print('What are the skills you\'re not familiar with?')
print('When you\'re done, press ENTER:')
unfamiliar_skills = []
skill = input('>>')
while(skill != ''):
    unfamiliar_skills.append(skill)
    skill = input('>>')

print(f'Filtering out {unfamiliar_skills} from job search')


def find_jobs(f, version):
    html_text = requests.get('https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=Python&txtLocation=').text
    soup = BeautifulSoup(html_text, 'lxml')

    jobs = soup.find_all('li', class_="clearfix job-bx wht-shd-bx")
    for job in jobs:
        published_date = job.find('span', class_="sim-posted").span.text.strip()
        if 'few' in published_date:
            company_name = job.find('h3', class_="joblist-comp-name").text.replace(' ', '').strip()
            skills_needed = job.find('span', class_="srp-skills").text.replace(' ', '').strip()

            skills = skills_needed.split(',')
            if(len(set(skills) & set(unfamiliar_skills)) == 0):

                # the link to the full-description of the job is under header-->h2-->a tag
                # with the link being under the "href"
                more_info = job.header.h2.a['href']

                # put the job information every 30 min in a .txt file
                # formatted as jobs_version{version}.txt
                f.write(f"Company Name: {company_name} \n")
                f.write(f"Required Skills: {skills_needed} \n")
                f.write(f"More Information: {more_info} \n")
                f.write('\n')

    print(f'File Saved: jobs_version{version}.txt')


if __name__ == "__main__":
    version = 1
    while True:
        with open(f'jobs_pool/jobs_version{version}.txt', 'w') as f:
            find_jobs(f, version)
        # run every 30 minutes
        sleep_time = 30
        print(f"Waiting for {sleep_time} minutes ...")
        time.sleep(sleep_time * 60)
        version += 1